In [1]:
%run "../00_project_config.ipynb"

import pathlib as pl
import pandas as pd

annotated_files_table = pl.Path(
    "/home/ebertp/work/code/cubi/project-run-hgsvc-hybrid-assemblies/annotations/autogen",
    "annotated_files.tsv"
)

annotated_files = pd.read_csv(annotated_files_table, header=0, sep="\t", comment="#")

DUMP_BATCH_NUM = 2

if DUMP_BATCH_NUM == 1:
    out_batch_summary_table = PROJECT_BASE.joinpath(
        "annotations", "data_freezes", "HGSVC3_data_batch1_file_details.tsv"
    )
    read_stats_file = PROJECT_DATA_ROOT.joinpath(
        "2023_batch_data_tables", "hgsvc3_batch1_read_stats.tsv"
    )
elif DUMP_BATCH_NUM == 2:
    out_batch_summary_table = PROJECT_BASE.joinpath(
        "annotations", "data_freezes", "HGSVC3_data_batch2_file_details.tsv"
    )
    read_stats_file = PROJECT_DATA_ROOT.joinpath(
        "2023_batch_data_tables", "hgsvc3_batch2_read_stats.tsv"
    )
elif DUMP_BATCH_NUM == 3:
    out_batch_summary_table = PROJECT_BASE.joinpath(
        "annotations", "data_freezes", "HGSVC3_data_batch3_file_details.tsv"
    )
    read_stats_file = PROJECT_DATA_ROOT.joinpath(
        "2023_batch_data_tables", "hgsvc3_batch3_read_stats.tsv"
    )
else:
    raise

    
add_batch_column = DUMP_BATCH_NUM
read_stats = pd.read_csv(read_stats_file, header=0, sep="\t")
    
file_sources = []
for row in read_stats.itertuples(index=True):
    if row.source == "all":
        file_sources.append("full_dataset")
        continue
    short_source = row.source.split(".")[0]
    matched = annotated_files.loc[annotated_files["file_name"].str.contains(short_source), :]
    if matched.empty:
        raise ValueError(short_source)
    if matched.shape[0] > 1:
        raise ValueError(f"{short_source} - {matched}")

    file_path = matched["file_rel_path"].values[0]
    if row.sample == "NA24385":
        # special case: external data for HG002 need to show
        # more of the path for proper source identification
        if row.read_type == "hifi":
            file_path = file_path.split("/", 1)[-1]
        else:
            # no other public source available - ?
            file_name = file_path.split("/")[-1]
            assert "03_08_22_R941_HG002" in file_name
            assert "Guppy_6.0.6_prom_sup" in file_name
            globus_endpoint = "9db1f0a6-a05a-11ea-8f06-0a21f750d19b"
            file_path = f"Globus|{globus_endpoint}|/HG002-T2T/sequencing/ont/{file_name}"
    else:
        file_path = file_path.split("/", 3)[-1]
    file_sources.append(file_path)
    
file_sources = pd.DataFrame(file_sources, index=read_stats.index, columns=["rel_path"])

read_stats = pd.concat([read_stats, file_sources], axis=1, ignore_index=False)
read_stats["batch_number"] = add_batch_column
if "sample_batch" in read_stats.columns:
    read_stats.drop("sample_batch", axis=1, inplace=True)

samples_in_batch = read_stats["sample"].nunique()

with open(out_batch_summary_table, "w") as dump:
    _ = dump.write("# AUTOGEN TABLE - DO NOT EDIT\n")
    _ = dump.write(f"# {TODAY}\n")
    _ = dump.write(f"# Samples in batch: {samples_in_batch}\n")
    read_stats.to_csv(dump, sep="\t", header=True, index=False)

print(f"Samples in batch number {add_batch_column}:")
print(samples_in_batch)

Samples in batch number 2:
16
